In [22]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

# define hyperparameters
batch_size = 128
learning_rate = 1e-2
num_epochs = 40

# download MNIST dataset
train_dataset = datasets.FashionMNIST(
    root='./datasets', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.FashionMNIST(
    root='./datasets', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f'size of training set is: {len(train_loader.dataset)}')
print(f'size of testing set is: {len(test_loader.dataset)}')

# define model: a convolution neural network
class ConvolutionNN(nn.Module):
    def __init__(self, in_dim, n_class):
        super(ConvolutionNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.ReLU(True), 
            nn.MaxPool2d(2, 2)
        )
        self.fullyc = nn.Sequential(
            nn.Linear(400, 120), 
            nn.Linear(120, 84), 
            nn.Linear(84, n_class)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fullyc(x)
        return x
        
model = ConvolutionNN(1, 10)
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


size of training set is: 60000
size of testing set is: 10000


In [23]:
for epoch in range(num_epochs):
    # train model
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss
        _, predict = torch.max(out, 1)
        running_acc += (predict == label).float().mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epochs, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
       
    # test model
    model.eval()
    eval_loss = 0
    eval_acc = 0
    for data in test_loader:
        img, label = data
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        with torch.no_grad():
            out = model(img)
            loss = criterion(out, label)
        eval_loss += loss
        _, predict = torch.max(out, 1)
        eval_acc += (predict == label).float().mean()

    print(f'Test Loss: {eval_loss/len(test_loader):.6f}, Acc: {eval_acc/len(test_loader):.6f}')

[1/40] Loss: 2.274930, Acc: 0.168724
Finish 1 epochs, Loss: 2.124444, Acc: 0.266613
Test Loss: 1.273697, Acc: 0.563489
[2/40] Loss: 0.982874, Acc: 0.630573
Finish 2 epochs, Loss: 0.930965, Acc: 0.647333
Test Loss: 0.843276, Acc: 0.664854
[3/40] Loss: 0.769929, Acc: 0.705287
Finish 3 epochs, Loss: 0.750824, Acc: 0.713897
Test Loss: 0.704102, Acc: 0.724288
[4/40] Loss: 0.676735, Acc: 0.742813
Finish 4 epochs, Loss: 0.664856, Acc: 0.746835
Test Loss: 0.646592, Acc: 0.751582
[5/40] Loss: 0.621989, Acc: 0.764167
Finish 5 epochs, Loss: 0.610572, Acc: 0.769656
Test Loss: 0.595420, Acc: 0.782140
[6/40] Loss: 0.579779, Acc: 0.784427
Finish 6 epochs, Loss: 0.570135, Acc: 0.788724
Test Loss: 0.564803, Acc: 0.793809
[7/40] Loss: 0.544842, Acc: 0.800000
Finish 7 epochs, Loss: 0.537574, Acc: 0.801667
Test Loss: 0.535991, Acc: 0.805182
[8/40] Loss: 0.515358, Acc: 0.811901
Finish 8 epochs, Loss: 0.510261, Acc: 0.813938
Test Loss: 0.514129, Acc: 0.815170
[9/40] Loss: 0.494883, Acc: 0.820365
Finish 9 ep